<center>

# **Análisis de las jugadas 2**

</center>

___

In [52]:
# Instalar las librerias para el analisis de las partidas
! pip install pandas

You should consider upgrading via the 'F:\Cosas\Lenguajes y Programacion\Python\python.exe -m pip install --upgrade pip' command.


In [53]:
import pandas as pd
import os

#### Se importa el archivo y se crea el dataFrame

In [54]:
path_archivo = os.path.join(os.path.realpath('..'), '..', 'registros', 'datos_de_jugadas.csv')
try:
    data_set = pd.read_csv(path_archivo, encoding='utf-8')
except FileNotFoundError:
    print('No se encontró el registro de jugadas')

In [55]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestamp        282 non-null    float64
 1   id               282 non-null    object 
 2   evento           282 non-null    object 
 3   nombre_jugador   282 non-null    object 
 4   edad_jugador     282 non-null    int64  
 5   genero_jugador   282 non-null    object 
 6   estado           222 non-null    object 
 7   texto_ingresado  175 non-null    object 
 8   respuesta        212 non-null    object 
 9   nivel            282 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 22.2+ KB


# Estadísticas

### Mejor Tiempo de Respuesta Correcta

* Tomo las filas que me sirven para hacer los cálculos.

In [56]:
jugadas = data_set[(data_set["evento"] == "intento") | (data_set["evento"] == "inicio_partida")]

* Duplico el dataFrame, uno de sin la primer fila y el otro sin la última. Lugo resto para obtener la duración de la respuesta.

In [57]:
last = jugadas.index[len(jugadas.index) - 1]
jugadas_resta = jugadas.drop(last).reset_index()
jugadas = jugadas.drop(0).reset_index()

jugadas['timestamp'] = jugadas['timestamp'] - jugadas_resta['timestamp']

* Busco el que tenga menor tiempo y acomodo para mostrar.

In [58]:
jugadas = jugadas[(jugadas["evento"] == "intento") & (jugadas["estado"] == "ok")]
tiempo_min = jugadas['timestamp'].min()

jugadas = jugadas.rename(columns={'timestamp': 'tiempo'}).drop('index', axis='columns')
jugadas[jugadas['tiempo'] == tiempo_min]

,tiempo,id,evento,nombre_jugador,edad_jugador,genero_jugador,estado,texto_ingresado,respuesta,nivel
142,1.112036,eea14e95-9cbb-4e8d-99b4-86bc8773340b,intento,Trinity,55,Femenino,ok,Adam Hloušek,Adam Hloušek,Personalizado


### Promedio de tiempo de respuestas exitosas

* Tomo las filas que me sirven para hacer los cálculos.

In [59]:
jugadas = data_set[(data_set["evento"] == "intento") | (data_set["evento"] == "inicio_partida")]

* Duplico el dataFrame, uno de sin la primer fila y el otro sin la última. Lugo resto para obtener la duración de la respuesta.

In [60]:
last = jugadas.index[len(jugadas.index) - 1]
jugadas_resta = jugadas.drop(last).reset_index()
jugadas = jugadas.drop(0).reset_index()

jugadas['timestamp'] = jugadas['timestamp'] - jugadas_resta['timestamp']

* Filtro el dataFrame por respuesta correcta y obtengo el promedio de tiempo. Luego adapto para mostrar.

In [61]:
jugadas = jugadas[jugadas["estado"] == "ok"]
tiempo = jugadas['timestamp'].sum() / len(jugadas)

jugadas = jugadas.loc[ :0, ['timestamp']].rename(columns={'timestamp': 'promedio de tiempo'})
jugadas['promedio de tiempo'] = tiempo
jugadas

,promedio de tiempo
0,4.405802


### Promedio de tiempo de partida

* Hago dos variables, uno con el inicio de partida y otro con el fin.

In [62]:
partida = data_set[data_set["evento"] == "inicio_partida"].reset_index()
partida_fin = data_set[data_set["evento"] == "fin"].reset_index()

* Obtengo el promedio del tiempo de partida.

In [63]:
partida['timestamp'] = partida_fin['timestamp'] - partida['timestamp']
tiempo = partida['timestamp'].sum() / len(partida)

* Acomodo para presentar el resultado.

In [64]:
partida = partida.loc[ :0, ['timestamp']].rename(columns={'timestamp': 'promedio de tiempo'})
partida['promedio de tiempo'] = tiempo
partida

,promedio de tiempo
0,28.388803


### Porcentaje de aciertos por usuario

* Filtro el dataFrame por las filas que necesito y obtengo cuantas veces aparece el mismo usuario.

In [65]:
jugadas = data_set[data_set["evento"] == "intento"]
jugadores = jugadas["nombre_jugador"].value_counts()

* Genero el dataFrame para los resultados.

In [66]:
res = pd.DataFrame({'jugador' : [], 'aciertos': [], 'errores': [], 'porcentaje de aciertos' : []})

* Para cada usuario guardo en los resultados la cantidad de aciertos y errores, además del porcentaje de acierto.

In [67]:
for x in jugadores.keys():
    jugador = jugadas[jugadas['nombre_jugador'] == x]['estado'].value_counts()
    if 'ok' in jugador:
        aciertos = jugador['ok']
    else:
        aciertos = 0
    if 'error' in jugador:
        errores = jugador['error']
    fila = pd.DataFrame({'jugador' : [x], 'aciertos': [aciertos], 'errores': errores, 
            'porcentaje de aciertos' : [f'{int(aciertos / jugadores[x] * 100)}%']})
    res = pd.concat([res, fila], ignore_index=True)
res = res.astype({'aciertos': 'int64', 'errores': 'int64'})

res

,jugador,aciertos,errores,porcentaje de aciertos
0,NEO,10,36,21%
1,Trinity,9,36,20%
2,Yoda,9,24,27%
3,Yayo,10,21,32%
4,El Jose,2,7,22%
5,kaggel,1,5,16%
6,Amelia,0,5,0%


### Cantidad de tarjetas para las que el usuarie no dió respuesta (timeout).

* Obtengo la cantidad de veces que un jugador no respondió y el total de jugadores.

In [68]:
jugadas = data_set[data_set["evento"] == "timeout"]["nombre_jugador"].value_counts()
jugadores = data_set["nombre_jugador"].value_counts().keys()

* Adapto el dataFrame para mostrar.

In [69]:
jugadas = pd.DataFrame(jugadas).reset_index()
jugadas.columns = ['jugador', 'Cantidad de tarjetas sin responder']
jugadas

,jugador,Cantidad de tarjetas sin responder
0,Yayo,5
1,NEO,3
2,Trinity,2
3,kaggel,2


### Cantidad de tarjetas en las que el usuarie dió una respuesta errónea.

* Obtengo la cantidad de veces que un jugador respondió mal y el total de jugadores.

In [70]:
jugadas = data_set[(data_set["evento"] == "intento") & (data_set["estado"] == "error")]["nombre_jugador"].value_counts()
jugadores = data_set["nombre_jugador"].value_counts().keys()

* Adapto el dataFrame para mostrar.

In [71]:
jugadas = pd.DataFrame(jugadas).reset_index()
jugadas.columns = ['jugador', 'Cantidad de respuestas erroneas']
jugadas

,jugador,Cantidad de respuestas erroneas
0,NEO,36
1,Trinity,36
2,Yoda,24
3,Yayo,21
4,El Jose,7
5,Amelia,5
6,kaggel,5
